In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from  IPython import display
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf

!rm -rf ./logs/ 

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
higgs_path = tf.keras.utils.get_file('HIGGSSmall.csv.gz', 'https://github.com/PacktWorkshops/The-Reinforcement-Learning-Workshop/blob/master/Chapter03/Dataset/HIGGSSmall.csv.gz?raw=true')

In [3]:
N_TEST = int(1e3)
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(N_TRAIN)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_FEATURES = 28

ds = tf.data.experimental.CsvDataset(higgs_path,[float(),]*(N_FEATURES+1), compression_type="GZIP")

def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:],1)
    return features, label

packed_ds = ds.batch(N_TRAIN).map(pack_row).unbatch()

In [4]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
test_ds = packed_ds.skip(N_VALIDATION).take(N_TEST).cache()
train_ds = packed_ds.skip(N_VALIDATION+N_TEST).take(N_TRAIN).cache()

test_ds = test_ds.batch(BATCH_SIZE)
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

In [5]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def compile_and_fit(model, name, max_epochs=3000):
    
    optimizer = tf.keras.optimizers.Adam(lr_schedule)
    
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[
                    tf.keras.losses.BinaryCrossentropy(
                        from_logits=True, name='binary_crossentropy'),
                    'accuracy'])

    model.summary()

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    
    history = model.fit(train_ds,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        epochs=max_epochs,
                        validation_data=validate_ds,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
                                   tensorboard_callback],
                        verbose=2)
    return history

In [7]:
regularization_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu', input_shape=(N_FEATURES,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

compile_and_fit(regularization_model, "regularizers/regularization", max_epochs=9000)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               14848     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               2

Epoch 42/9000
20/20 - 0s - loss: 0.7010 - binary_crossentropy: 0.6188 - accuracy: 0.6244 - val_loss: 0.6822 - val_binary_crossentropy: 0.6009 - val_accuracy: 0.6320
Epoch 43/9000
20/20 - 1s - loss: 0.7048 - binary_crossentropy: 0.6242 - accuracy: 0.6176 - val_loss: 0.6812 - val_binary_crossentropy: 0.6015 - val_accuracy: 0.6260
Epoch 44/9000
20/20 - 0s - loss: 0.6995 - binary_crossentropy: 0.6205 - accuracy: 0.6207 - val_loss: 0.6735 - val_binary_crossentropy: 0.5953 - val_accuracy: 0.6460
Epoch 45/9000
20/20 - 0s - loss: 0.6985 - binary_crossentropy: 0.6210 - accuracy: 0.6238 - val_loss: 0.6745 - val_binary_crossentropy: 0.5979 - val_accuracy: 0.6250
Epoch 46/9000
20/20 - 0s - loss: 0.6922 - binary_crossentropy: 0.6163 - accuracy: 0.6297 - val_loss: 0.6763 - val_binary_crossentropy: 0.6011 - val_accuracy: 0.6210
Epoch 47/9000
20/20 - 1s - loss: 0.6944 - binary_crossentropy: 0.6200 - accuracy: 0.6211 - val_loss: 0.6720 - val_binary_crossentropy: 0.5984 - val_accuracy: 0.6240
Epoch 48/9

Epoch 92/9000
20/20 - 0s - loss: 0.6388 - binary_crossentropy: 0.6067 - accuracy: 0.6491 - val_loss: 0.6182 - val_binary_crossentropy: 0.5864 - val_accuracy: 0.6780
Epoch 93/9000
20/20 - 1s - loss: 0.6347 - binary_crossentropy: 0.6030 - accuracy: 0.6459 - val_loss: 0.6165 - val_binary_crossentropy: 0.5850 - val_accuracy: 0.6740
Epoch 94/9000
20/20 - 0s - loss: 0.6354 - binary_crossentropy: 0.6041 - accuracy: 0.6484 - val_loss: 0.6163 - val_binary_crossentropy: 0.5853 - val_accuracy: 0.6580
Epoch 95/9000
20/20 - 1s - loss: 0.6368 - binary_crossentropy: 0.6060 - accuracy: 0.6453 - val_loss: 0.6195 - val_binary_crossentropy: 0.5889 - val_accuracy: 0.6230
Epoch 96/9000
20/20 - 0s - loss: 0.6363 - binary_crossentropy: 0.6059 - accuracy: 0.6456 - val_loss: 0.6189 - val_binary_crossentropy: 0.5888 - val_accuracy: 0.6630
Epoch 97/9000
20/20 - 0s - loss: 0.6347 - binary_crossentropy: 0.6047 - accuracy: 0.6455 - val_loss: 0.6149 - val_binary_crossentropy: 0.5852 - val_accuracy: 0.6710
Epoch 98/9

Epoch 142/9000
20/20 - 0s - loss: 0.6199 - binary_crossentropy: 0.5959 - accuracy: 0.6600 - val_loss: 0.6083 - val_binary_crossentropy: 0.5843 - val_accuracy: 0.6620
Epoch 143/9000
20/20 - 1s - loss: 0.6181 - binary_crossentropy: 0.5941 - accuracy: 0.6597 - val_loss: 0.5939 - val_binary_crossentropy: 0.5700 - val_accuracy: 0.6670
Epoch 144/9000
20/20 - 1s - loss: 0.6181 - binary_crossentropy: 0.5940 - accuracy: 0.6572 - val_loss: 0.6001 - val_binary_crossentropy: 0.5761 - val_accuracy: 0.6680
Epoch 145/9000
20/20 - 0s - loss: 0.6204 - binary_crossentropy: 0.5964 - accuracy: 0.6607 - val_loss: 0.5998 - val_binary_crossentropy: 0.5759 - val_accuracy: 0.6760
Epoch 146/9000
20/20 - 0s - loss: 0.6194 - binary_crossentropy: 0.5955 - accuracy: 0.6535 - val_loss: 0.5988 - val_binary_crossentropy: 0.5749 - val_accuracy: 0.6780
Epoch 147/9000
20/20 - 1s - loss: 0.6200 - binary_crossentropy: 0.5961 - accuracy: 0.6615 - val_loss: 0.5981 - val_binary_crossentropy: 0.5741 - val_accuracy: 0.6720
Epoc

Epoch 192/9000
20/20 - 0s - loss: 0.6108 - binary_crossentropy: 0.5850 - accuracy: 0.6657 - val_loss: 0.5908 - val_binary_crossentropy: 0.5650 - val_accuracy: 0.6840
Epoch 193/9000
20/20 - 0s - loss: 0.6136 - binary_crossentropy: 0.5878 - accuracy: 0.6735 - val_loss: 0.5929 - val_binary_crossentropy: 0.5670 - val_accuracy: 0.6830
Epoch 194/9000
20/20 - 0s - loss: 0.6145 - binary_crossentropy: 0.5887 - accuracy: 0.6691 - val_loss: 0.5939 - val_binary_crossentropy: 0.5681 - val_accuracy: 0.6750
Epoch 195/9000
20/20 - 0s - loss: 0.6153 - binary_crossentropy: 0.5894 - accuracy: 0.6616 - val_loss: 0.5915 - val_binary_crossentropy: 0.5656 - val_accuracy: 0.6910
Epoch 196/9000
20/20 - 1s - loss: 0.6154 - binary_crossentropy: 0.5895 - accuracy: 0.6660 - val_loss: 0.5942 - val_binary_crossentropy: 0.5682 - val_accuracy: 0.6740
Epoch 197/9000
20/20 - 0s - loss: 0.6134 - binary_crossentropy: 0.5874 - accuracy: 0.6672 - val_loss: 0.5902 - val_binary_crossentropy: 0.5642 - val_accuracy: 0.6780
Epoc

...

20/20 - 0s - loss: 0.5824 - binary_crossentropy: 0.5420 - accuracy: 0.7051 - val_loss: 0.5788 - val_binary_crossentropy: 0.5384 - val_accuracy: 0.6970
Epoch 1290/9000
20/20 - 0s - loss: 0.5853 - binary_crossentropy: 0.5449 - accuracy: 0.7052 - val_loss: 0.5851 - val_binary_crossentropy: 0.5447 - val_accuracy: 0.6940
Epoch 1291/9000
20/20 - 0s - loss: 0.5871 - binary_crossentropy: 0.5467 - accuracy: 0.7017 - val_loss: 0.5811 - val_binary_crossentropy: 0.5407 - val_accuracy: 0.6960
Epoch 1292/9000
20/20 - 0s - loss: 0.5897 - binary_crossentropy: 0.5492 - accuracy: 0.7034 - val_loss: 0.5823 - val_binary_crossentropy: 0.5418 - val_accuracy: 0.6970
Epoch 1293/9000
20/20 - 1s - loss: 0.5879 - binary_crossentropy: 0.5475 - accuracy: 0.6976 - val_loss: 0.5762 - val_binary_crossentropy: 0.5357 - val_accuracy: 0.7050
Epoch 1294/9000
20/20 - 0s - loss: 0.5887 - binary_crossentropy: 0.5483 - accuracy: 0.6965 - val_loss: 0.5789 - val_binary_crossentropy: 0.5385 - val_accuracy: 0.7000
Epoch 1295/900

Epoch 1339/9000
20/20 - 0s - loss: 0.5854 - binary_crossentropy: 0.5444 - accuracy: 0.7081 - val_loss: 0.5767 - val_binary_crossentropy: 0.5357 - val_accuracy: 0.7030
Epoch 1340/9000
20/20 - 0s - loss: 0.5846 - binary_crossentropy: 0.5436 - accuracy: 0.7060 - val_loss: 0.5805 - val_binary_crossentropy: 0.5395 - val_accuracy: 0.7070
Epoch 1341/9000
20/20 - 1s - loss: 0.5840 - binary_crossentropy: 0.5429 - accuracy: 0.7042 - val_loss: 0.5731 - val_binary_crossentropy: 0.5320 - val_accuracy: 0.7040
Epoch 1342/9000
20/20 - 0s - loss: 0.5872 - binary_crossentropy: 0.5461 - accuracy: 0.6969 - val_loss: 0.5818 - val_binary_crossentropy: 0.5407 - val_accuracy: 0.7060
Epoch 1343/9000
20/20 - 0s - loss: 0.5897 - binary_crossentropy: 0.5487 - accuracy: 0.7015 - val_loss: 0.5805 - val_binary_crossentropy: 0.5395 - val_accuracy: 0.7040
Epoch 1344/9000
20/20 - 1s - loss: 0.5857 - binary_crossentropy: 0.5447 - accuracy: 0.7035 - val_loss: 0.5773 - val_binary_crossentropy: 0.5363 - val_accuracy: 0.709

20/20 - 0s - loss: 0.5843 - binary_crossentropy: 0.5428 - accuracy: 0.7049 - val_loss: 0.5822 - val_binary_crossentropy: 0.5407 - val_accuracy: 0.7070
Epoch 1389/9000
20/20 - 1s - loss: 0.5854 - binary_crossentropy: 0.5439 - accuracy: 0.7061 - val_loss: 0.5825 - val_binary_crossentropy: 0.5410 - val_accuracy: 0.7040
Epoch 1390/9000
20/20 - 0s - loss: 0.5863 - binary_crossentropy: 0.5449 - accuracy: 0.7044 - val_loss: 0.5786 - val_binary_crossentropy: 0.5372 - val_accuracy: 0.7030
Epoch 1391/9000
20/20 - 0s - loss: 0.5826 - binary_crossentropy: 0.5412 - accuracy: 0.7080 - val_loss: 0.5822 - val_binary_crossentropy: 0.5408 - val_accuracy: 0.7080
Epoch 1392/9000
20/20 - 1s - loss: 0.5833 - binary_crossentropy: 0.5418 - accuracy: 0.7042 - val_loss: 0.5828 - val_binary_crossentropy: 0.5413 - val_accuracy: 0.7210
Epoch 1393/9000
20/20 - 0s - loss: 0.5827 - binary_crossentropy: 0.5412 - accuracy: 0.7001 - val_loss: 0.5880 - val_binary_crossentropy: 0.5465 - val_accuracy: 0.7020
Epoch 1394/900

Epoch 1438/9000
20/20 - 0s - loss: 0.5829 - binary_crossentropy: 0.5409 - accuracy: 0.7078 - val_loss: 0.5814 - val_binary_crossentropy: 0.5394 - val_accuracy: 0.7020
Epoch 1439/9000
20/20 - 0s - loss: 0.5832 - binary_crossentropy: 0.5411 - accuracy: 0.7048 - val_loss: 0.5831 - val_binary_crossentropy: 0.5411 - val_accuracy: 0.7040
Epoch 1440/9000
20/20 - 0s - loss: 0.5866 - binary_crossentropy: 0.5446 - accuracy: 0.7042 - val_loss: 0.5813 - val_binary_crossentropy: 0.5393 - val_accuracy: 0.7020
Epoch 1441/9000
20/20 - 1s - loss: 0.5813 - binary_crossentropy: 0.5393 - accuracy: 0.7075 - val_loss: 0.5836 - val_binary_crossentropy: 0.5415 - val_accuracy: 0.7060
Epoch 1442/9000
20/20 - 0s - loss: 0.5853 - binary_crossentropy: 0.5433 - accuracy: 0.7027 - val_loss: 0.5835 - val_binary_crossentropy: 0.5415 - val_accuracy: 0.6990
Epoch 1443/9000
20/20 - 0s - loss: 0.5860 - binary_crossentropy: 0.5439 - accuracy: 0.7024 - val_loss: 0.5854 - val_binary_crossentropy: 0.5433 - val_accuracy: 0.690

20/20 - 0s - loss: 0.5807 - binary_crossentropy: 0.5381 - accuracy: 0.7056 - val_loss: 0.5880 - val_binary_crossentropy: 0.5454 - val_accuracy: 0.6940
Epoch 1488/9000
20/20 - 1s - loss: 0.5842 - binary_crossentropy: 0.5415 - accuracy: 0.7088 - val_loss: 0.5790 - val_binary_crossentropy: 0.5362 - val_accuracy: 0.7110
Epoch 1489/9000
20/20 - 1s - loss: 0.5813 - binary_crossentropy: 0.5386 - accuracy: 0.7068 - val_loss: 0.5773 - val_binary_crossentropy: 0.5346 - val_accuracy: 0.7100
Epoch 1490/9000
20/20 - 0s - loss: 0.5857 - binary_crossentropy: 0.5430 - accuracy: 0.7053 - val_loss: 0.5842 - val_binary_crossentropy: 0.5415 - val_accuracy: 0.7080
Epoch 1491/9000
20/20 - 0s - loss: 0.5801 - binary_crossentropy: 0.5374 - accuracy: 0.7063 - val_loss: 0.5885 - val_binary_crossentropy: 0.5458 - val_accuracy: 0.6910
Epoch 1492/9000
20/20 - 0s - loss: 0.5852 - binary_crossentropy: 0.5424 - accuracy: 0.7056 - val_loss: 0.5854 - val_binary_crossentropy: 0.5426 - val_accuracy: 0.7120
Epoch 1493/900

In [8]:
test_accuracy = tf.keras.metrics.Accuracy()

for (features, labels) in test_ds:
    logits = regularization_model(features)
    probabilities = tf.keras.activations.sigmoid(logits)
    predictions = 1*(probabilities.numpy() > 0.5)
    test_accuracy(predictions, labels)
    regularization_model_accuracy = test_accuracy.result()

print("Test set accuracy: {:.3%}".format(regularization_model_accuracy))

Test set accuracy: 69.300%


In [9]:
%tensorboard --logdir logs/fit